# Using Google's Inception-ResNet v2 to Classify Staircases

## Import Modules and Dataset

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pickle, os
import tensorflow as tf
import keras
import time
from keras.utils import plot_model
from keras import layers

#Static Variables
test_data_percentage = 0.1
class_names = ["curved", "negative", "straight"]
epochs = 30

#Tensorboard
!rm logs/Stairs/*
NAME = "Stairs"
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/{}".format(NAME))

#Load X and Y Lists from Pickle Files
x_bfile = open('x.pickle', 'rb')
x_data = pickle.load(x_bfile)/255.0

y_bfile = open('y.pickle', 'rb')
y_data_int = pickle.load(y_bfile)
y_data = keras.utils.to_categorical(y_data_int, num_classes=3)

#Extract Test Data
no_train = int(len(x_data)*(1-test_data_percentage))
img_size = len(x_data[0])
x_train = x_data[:no_train]
y_train = y_data[:no_train]
y_train_int = y_data_int[:no_train]
x_test = x_data[no_train:]
y_test = y_data[no_train:]
y_test_int = y_data_int[no_train:]

#Configing to release GPU memory upon completion
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

print("Dataset Parsed")
print("Image Size:", img_size)

rm: cannot remove 'logs/Stairs/*': No such file or directory


MemoryError: 

### Training Data Visalisation
Plots an image and its corresponding label to check if the data has been processed correctly.

In [ ]:
img_to_show = 301

x_train_img = np.resize(x_train,(len(x_train), img_size, img_size))
x_train_img  = x_train_img.astype("float32")
plt.imshow(x_train_img[img_to_show], cmap ='gray')
plt.xlabel(class_names[y_train_int[img_to_show]], fontsize=18)
print("x_train Shape:", x_train.shape)
print("y_train Shape:", y_train.shape)
print("y_train:\n", y_train)

## Create, Compile and Train Model

In [ ]:
#Manual Validation Split
num_val = int(len(x_train)*0.1)
x_val = x_train[:num_val]
y_val = y_train[:num_val]
x_train = x_train[num_val:]
y_train = y_train[num_val:]

#Training Datagen
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_datagen.fit(x_train)
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)

#Validation Datagen
val_datagen = keras.preprocessing.image.ImageDataGenerator() #no data augmentation
val_generator = val_datagen.flow(x_val, y_val, batch_size=32)
val_steps = int(num_val/32)

In [ ]:
############ Create Model Using Inception-ResNet v2 (Keras Applications)
from keras.applications.inception_resnet_v2 import InceptionResNetV2

img_input = layers.Input(shape=(img_size,img_size,1))
img_conc = layers.Concatenate()([img_input, img_input, img_input])    

#Preparing the Base Model (299x299x3 input)
base_model = InceptionResNetV2(input_tensor=img_conc,weights='imagenet', include_top=False)

#Modifying the End of the Base Model
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(3, activation='softmax')(x)

#Putting the model together 
model = keras.models.Model(inputs=base_model.input, outputs=x)

# 261 is the end of the 10th a block: block35_10_ac
# 275 is the end of the a reduction block
# 595 is the end of the 20th b block: block17_20_ac
# 618 is the end of the b reduction block: mixed_7a
# 777 is the end of the 10th c block: block8_10

# for i, layer in enumerate(base_model.layers):
#    print(i, layer.shape)

# print(model.summary())
# plot_model(model, to_file='model.png', dpi=200, show_shapes=True, show_layer_names=False)


############ Training
epoch_new = 5 #5
epoch_bc= 20 #20

#Train New Layers first
print("Training NEW LAYERS")
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(x_train, y_train, epochs=epoch_new, validation_split=0.1)
model.fit_generator(train_generator, validation_data=val_generator,steps_per_epoch=len(x_train) / 32,validation_steps=val_steps, epochs=epoch_new)

#Train b and c blocks
print("Training B and C")
for layer in model.layers[:275]: #275
    layer.trainable=False
for layer in model.layers[275:]:
    layer.trainable=True

from keras.optimizers import Adam

_optimizer = Adam(lr=0.0001)
model.compile(optimizer=_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#history = model.fit(x_train, y_train, epochs=epoch_bc, validation_split=0.1, callbacks=[tensorboard])
history = model.fit_generator(train_generator, validation_data=val_generator,steps_per_epoch=len(x_train) / 32, validation_steps=val_steps, epochs=epoch_bc)
print("Training Finished")

In [ ]:
#Save the Model
model.save('stairs_model.h5')
print("Model Saved")
sess.close()

### Loading a Model

In [ ]:
#Load Model
model = keras.models.load_model('stairs_model.h5')
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

print("Model Loaded")

## Evaluate Model

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("\nAccuracy Stats")
print("Train Accuracy:", round(history.history['acc'][-1]*10000)/100, "%")
print("Validation Accuracy:", round(history.history['val_acc'][-1]*10000)/100, "%")
print("Test Accuracy:", round(test_acc*10000)/100, "%")

print("\nLoss Stats")
print("Train Loss:", round(history.history['loss'][-1]*1000)/1000)
print("Validation Loss:", round(history.history['val_loss'][-1]*1000)/1000)
print("Test Loss:", round(test_loss*1000)/1000)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Cross Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Cross Validation'], loc='upper left')
plt.show()

## Predictions

In [ ]:
predictions = model.predict(x_test)
print("Generated Predictions for Test Data")
print("Predictions Shape:",predictions.shape)

### Showing Predictions
predict_img(i) shows predicted and actual class of image i in the test set

In [ ]:
print("Number of Test Images", len(x_test))
x_test_img = np.resize(x_test, (len(x_test), img_size, img_size))
x_test_img = x_test_img.astype('float32')
def predict_img(i):
    print(predictions[1])
    plt.imshow(x_test_img[i], cmap="gray")
    plt.xlabel("Prediction: [" + str(round(predictions[i][0]*100)/100.0) + "][" + str(round(predictions[i][1]*100)/100.0) + "][" + str(round(predictions[i][2]*100)/100.0) + "] - "+ class_names[np.argmax(predictions[i])]+ ", Actual Class: " + class_names[y_test_int[i]], fontsize=18)

In [ ]:
predict_img(93)